In [3]:
stableStringify = (val) => {
  if (Array.isArray(val)) return `[${val.map(stableStringify).join(',')}]`;
  if (val && typeof val === 'object' && !(val instanceof Date)) {
    return `{${Object.keys(val).sort().map(key => `${JSON.stringify(key)}:${stableStringify(val[key])}`).join(',')}}`;
  }
  return JSON.stringify(val);
};
deepEqual = (a, b) => {
  if (a === b) return true;
  const aObj = a && typeof a === 'object';
  const bObj = b && typeof b === 'object';
  if (!aObj && !bObj) return Object.is(a, b);
  return stableStringify(a) === stableStringify(b);
};
formatVal = (val) => (typeof val === 'string' ? `'${val}'` : stableStringify(val));
assert = (fn, expected, label) => {
  try {
    const actual = fn();
    const pass = deepEqual(actual, expected);
    if (pass) {
      console.log('✓', label, '>>', formatVal(actual));
    } else {
      console.error('✗', label, '>> expected:', formatVal(expected), 'actual:', formatVal(actual));
    }
  } catch (err) {
    console.error('✗', label, '>> threw:', err);
  }
};

[Function: assert]

In [4]:
nameCache = new Map()
kebabToCamel = s => {
  if (!s) return s
  let p = s.indexOf('-')
  if (p === -1) return s
  if (nameCache.has(s)) return nameCache.get(s)
  let result = s.slice(0, p)
  while (p != -1 && ++p < s.length) {
    if (s[p] == '-') continue;
    result += s[p].toUpperCase()
    if (++p < s.length)
      result += s.slice(p, (p = s.indexOf('-', p)) == -1 ? s.length : p)
  }
  nameCache.set(s, result)
  return result
}

assert(() => kebabToCamel('foo-bar'), 'fooBar', 'basic case');
assert(() => kebabToCamel('-bar'), 'Bar', 'lead single');
assert(() => kebabToCamel('bar-'), 'bar', 'trail single');
assert(() => kebabToCamel('multi-part-key'), 'multiPartKey', 'multi part');
assert(() => kebabToCamel('-'), '', 'single dash');
assert(() => kebabToCamel('--'), '', 'multi dashes only');
assert(() => kebabToCamel('--leading--dashes'), 'LeadingDashes', 'leading dashes');
assert(() => kebabToCamel('trailing--dashes-'), 'trailingDashes', 'trailing dashes');

✓ basic case >> 'fooBar'
✓ lead single >> 'Bar'
✓ trail single >> 'bar'
✓ multi part >> 'multiPartKey'
✓ single dash >> ''
✓ multi dashes only >> ''
✓ leading dashes >> 'LeadingDashes'
✓ trailing dashes >> 'trailingDashes'


In [5]:
// Returns the index of the first found char (from chars) in the s, or -1 if none found,
// then you can check s[retunedIndex] to see which char it is.
indexFirst = (s, chars, pos = 0) => {
  let i, first = s.length
  for (let c of chars) if ((i = s.indexOf(c, pos)) != -1 && i < first) first = i
  return first === s.length ? -1 : first
}

assert(() => indexFirst('abcdefg', ['x', 'y', 'z']), -1, 'none found');
assert(() => indexFirst('abcdefg', ['x', 'c', 'z']), 2, 'one found');
assert(() => indexFirst('abcdefgabc', ['a', 'b', 'c'], 3), 7, 'multiple found with pos');
assert(() => indexFirst('abcdefg', ['a'], 10), -1, 'pos out of range');

✓ none found >> -1
✓ one found >> 2
✓ multiple found with pos >> 7
✓ pos out of range >> -1


In [6]:
// @WIP using +#tpl-id instead of #tpl-id to be predictable in presence of other selectors
MOD = '^', TARG = ':', TRIG = '@', STAT = '?', ADD = '+', REM = '~'
ALL = [MOD, TARG, TRIG, STAT, ADD, REM]
MODS = [MOD]

// returns { [MOD]: mods, [TARG]: [targets [it, {[MOD]: [mods...]}]...],...}
parse = (n, p='data-'.length, it = ALL) => {
  let items = {}, mods = null
  while (p >= 0 && p < n.length) {
    if ((p = indexFirst(n, it, p)) == -1) { p = n.length; break }
    let t = n[p], name = ''
    if (++p < n.length) {
      let end = indexFirst(n, ALL, p)
      name = n.slice(p, p = end != -1 ? end : n.length)
    }

    let ts = items[t] ??= []
    if (t == MOD) {
      ts.push(name);
      if (p >= n.length || (it === MODS && n[p] != MOD)) return [items, p]
    } else if (p >= n.length || n[p] != MOD) {
      ts.push([name, null]);
    } else {
      [mods, p] = parse(n, p, MODS)
      ts.push([name, mods?.[MOD]])
    }
  }

  if (p < n.length) console.warn('Not everything is parsed "', n.slice(p), '" in', n)
  return [items, p]
}

assert(() => parse('data-def'), [{},8], 'empty')
assert(() => parse('data-sub:'), [{":":[["",null]]},9], 'single empty')
assert(() => parse('data-sub^mod'), [{"^":["mod"]},12], 'global mod')
assert(() => parse('data-sub^mod^'), [{"^":["mod",""]},13], '2 global mods')
assert(() => parse('data-sub^mod^@hey^foo:bar'), [{":":[["bar",null]],"@":[["hey",["foo"]]],"^":["mod",""]},25], '2 global mods and item with mod with item')
assert(() => parse('data-sub:xxx~'), [{":":[["xxx",null]],"~":[["",null]]},13], 'name and empty')
assert(() => parse('data-sub:xxx:'), [{":":[["xxx",null],["",null]]},13], 'single name')
assert(() => parse('data-sub::'), [{":":[["",null],["",null]]},10], '2 empties')
assert(() => parse('data-sub:foo^'), [{ ":": [["foo", [""]]] }, 13], 'name+empty mod')
assert(() => parse('data-sub:foo^^'), [{ ":": [["foo", ["", ""]]] }, 14], 'name+2 empty mods')
assert(() => parse('data-sub:foo-bar^bax.3'), [{ ":": [["foo-bar", ["bax.3"]]] }, 22], 'item^mod')
assert(() => parse('data-sub:foo-bar^bax.3@!something^nice'), [{ ":": [["foo-bar", ["bax.3"]]], "@": [["!something", ["nice"]]] }, 38], 'item^mod@item2^mod')

✓ empty >> [{},8]
✓ single empty >> [{":":[["",null]]},9]
✓ global mod >> [{"^":["mod"]},12]
✓ 2 global mods >> [{"^":["mod",""]},13]
✓ 2 global mods and item with mod with item >> [{":":[["bar",null]],"@":[["hey",["foo"]]],"^":["mod",""]},25]
✓ name and empty >> [{":":[["xxx",null]],"~":[["",null]]},13]
✓ single name >> [{":":[["xxx",null],["",null]]},13]
✓ 2 empties >> [{":":[["",null],["",null]]},10]
✓ name+empty mod >> [{":":[["foo",[""]]]},13]
✓ name+2 empty mods >> [{":":[["foo",["",""]]]},14]
✓ item^mod >> [{":":[["foo-bar",["bax.3"]]]},22]
✓ item^mod@item2^mod >> [{":":[["foo-bar",["bax.3"]]],"@":[["!something",["nice"]]]},38]


In [9]:

isObjEmpty = (o) => {
  for (const _ in o) return false
  return true
}

isProp = (n) => n.startsWith('#') || n.startsWith('.')

RETURN_THEN = [' ', '(', '{', ';', '[', '"', '\'', '\n', '\r', '\t']
normBody = (s) => {
  let body = String(s)
  var r = body.indexOf('return')
  return r != -1 && indexFirst(body, RETURN_THEN, r + 6) == r + 6 ? body : `return(${body})`
}

DM = {}

// - data-def='{foo: {bar: "hey"}, baz: 1}' // top level fields to signals
// - data-def:foo='{bar: "hey"}' // foo signal
// - data-def:foo:baz='`js expr ${42}`' // eval expr as Function body and set to all signals
// - data-def:foo='el.Value * dm.bar' // you may use other singals and element props
dSign = (el, aName, aVal) => {
  let [it, _p] = parse(aName)
  let tars = it[TARG]
  delete it[TARG]
  if (!isObjEmpty(it)) console.warn('Supports only targets but found:', aName)
  let val = null
  if (aVal) {
    try { val = Function('dm', 'el', normBody(aVal))(DM, el); }
    catch (e) { console.error('Eval error:', e.message, `>> ${aName}='${aVal}'`); return }
  }
  if (tars && tars.length > 0) {
    for (const [t, mods] of tars) {
      if (mods) console.warn('Mods are not supported:', mods, 'in', aName);
      if (isProp(t)) { console.warn('Prop targets are not supported:', t, 'in', aName); continue }
      DM[kebabToCamel(t)] = val
    }
  } else if (!val || typeof val !== 'object') {
    console.error('Attribute', aName, 'value should contain object with signal fields, but found', aVal)
  } else {
    for (const t in val)
      DM[kebabToCamel(t)] = val[t]
  }
}

D_SIGN = 'def';
DA = {
  [D_SIGN]: dSign,
  //   'data-sync': setupSync,
  //   'data-sub': setupSub,
  //   'data-class': setupClass,
  //   'data-disp': setupDisp,
  //   'data-dump': setupDump,
  //   'data-get': setupAction,
  //   'data-post': setupAction,
  //   'data-put': setupAction,
  //   'data-patch': setupAction,
  //   'data-delete': setupAction
}

_sign = (nam, val) => { DM = {}; DA[D_SIGN](null, nam, val); return DM }
assert(() => _sign('data-def', '{foo: {bar: "hey"}, baz: 1}'), { "baz": 1, "foo": { "bar": "hey" } }, '2 value signals')
assert(() => _sign('data-def:foo', '{bar: "hey"}'), { "foo": { "bar": "hey" } }, 'signal = value')
assert(() => _sign('data-def:foo-bar:baz'), { "baz": null, "fooBar": null }, 'signals')
assert(() => _sign('data-def:foo-bar:baz', '`Mamma Mia ${42}`'), { "baz": "Mamma Mia 42", "fooBar": "Mamma Mia 42" }, 'bonkers')

_sign = (el, nam, val) => { DM = {}; DA[D_SIGN](el, nam, val); return DM }
assert(() => _sign({ "name": "John" }, 'data-def:foo', '"Hey, " + el.name'), { "foo": "Hey, John" }, '2 value signals')

// quick check:
// fn_foo = (el) => { let n = el.name; return n }
// console.log(fn_foo({ "name": "John" }))
// fn_bar = (el) => { return (el.name) }
// console.log(fn_bar({ "name": "John" }))

✓ 2 value signals >> {"baz":1,"foo":{"bar":"hey"}}
✓ signal = value >> {"foo":{"bar":"hey"}}
✓ signals >> {"baz":null,"fooBar":null}
✓ bonkers >> {"baz":"Mamma Mia 42","fooBar":"Mamma Mia 42"}
✓ 2 value signals >> {"foo":"Hey, John"}


In [8]:
  compile = body => {
    if(fnCache.has(body)) return fnCache.get(body);
    let inner;
    try {
      // signature: (dm, el, ev, sg, detail)
      // dm is a plain object built from the internal signal Map (S), so expressions
      // can read values as `dm.foo` or `dm.user.name`.
      inner = new Function('dm','el','ev','sg','detail', `try{ return (${body}); }catch(e){ console.error(e); return undefined; }`);
    } catch(e) {
      console.error('[dmax] Failed to compile expression:', body.slice(0, 50) + (body.length > 50 ? '...' : ''), e);
      inner = () => undefined;
    }
    // Phase 1.4: Error boundary with expression context
    const wrapped = (dm, el, ev, sg, detail) => {
      try { 
        return inner(dm, el, ev, sg, detail); 
      } catch (e) { 
        console.error('[dmax] Expression failed:', body.slice(0, 60) + (body.length > 60 ? '...' : ''), e);
        return undefined; 
      }
    };
    fnCache.set(body, wrapped);
    return wrapped;
  };

[Function: compile]